# Agentic AI
Questo notebook contiene una serie di semplici esempi per usare le generative AI per fare agentic intelligenti.

## LLM Provider
Un provider è un servizio che fornisce modelli di linguaggio (LLM) per generare testo, rispondere a domande, tradurre lingue, e altro ancora.
Noi vedremo sia Ollama (riferimento per caricare *solo* modelli locali) e LM Studio (pensato per la gestione di modelli in locale e remoti e in generale per la prototipazione rapida di agenti AI).

### Ollama
È un servizio che è composto da:
- runtime per eseguire modelli di linguaggio in locale
- un'interfaccia a linea di comando (CLI) per gestire i modelli e
- un'interfaccia web per interagire con i modelli.
In automatico, carica e gestisce i modelli open cercando di ottimizzare le risorse del sistema (e.g., GPU, RAM, CPU).

In [1]:
import ollama
models = ollama.list().models
for model in models:
    print(model.model)

qwen3-vl:2b
gemma3:270m
qwen3:1.7b
llama3.2:latest
llama3.2:3b
qwen2.5:1.5b
qwen2.5:3b
smollm:latest
smollm:360m
mxbai-embed-large:latest
smollm:135m
all-minilm:latest


### Generazione
Per generare il testo con un modello, si usa la funzione `ollama.generate`.
Accetta come parametri il nome del modello e il prompt.

In [ ]:
ollama.generate("gemma3:270m", "Ciao, come stai?").response

### OpenAI compatibility
Ollama fornisce un'API compatibile con OpenAI, quindi è possibile usare librerie che supportano OpenAI per interagire con i modelli Ollama.

In [3]:
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="none" # Ollama non richiede API key
)
client.models.list()

SyncPage[Model](data=[Model(id='qwen3-vl:2b', created=1762677138, object='model', owned_by='library'), Model(id='gemma3:270m', created=1762623941, object='model', owned_by='library'), Model(id='qwen3:1.7b', created=1762620637, object='model', owned_by='library'), Model(id='llama3.2:latest', created=1762609374, object='model', owned_by='library'), Model(id='llama3.2:3b', created=1762609335, object='model', owned_by='library'), Model(id='qwen2.5:1.5b', created=1741084202, object='model', owned_by='library'), Model(id='qwen2.5:3b', created=1741078970, object='model', owned_by='library'), Model(id='smollm:latest', created=1741069260, object='model', owned_by='library'), Model(id='smollm:360m', created=1740861984, object='model', owned_by='library'), Model(id='mxbai-embed-large:latest', created=1740858009, object='model', owned_by='library'), Model(id='smollm:135m', created=1740855169, object='model', owned_by='library'), Model(id='all-minilm:latest', created=1740855088, object='model', own

In [5]:
client.completions.create(model="gemma3:270m", prompt="Ciao, come stai?").choices[0].text

"Ciao! Sto bene, grazie per avermi chiesto un'aiuta. Come stai oggi?\n"

## Langchain
LangChain è una libreria che facilita la costruzione di applicazioni basate su modelli di linguaggio.
Abbiamo visto OpenWebUI che ci permette di creare semplici chatbot con modelli locali.
Per fare agenti "custom" (workflow complessi principalmente) possiamo usare LangChain con modelli locali (tramite Ollama o LM Studio).
Langchain principalmente ha bisogno di un modello, noi ci baseremo su ChatOpenAI che supporta modelli di tipo chat.

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="qwen3:1.7b",
    base_url="http://localhost:11434/v1",
    # model="qwen/qwen3-vl-4b",
    # base_url="http://127.0.0.1:1234/v1",
    api_key="none"
)

`invoke` permette di invocare il modello con un singolo messaggio.

In [8]:
model.invoke("Che ore sono?").content

'Ciao! Sto cercando di capire che ora è, ma non ho accesso a dati real-time. Posso chiederti dove ti trovi? Così potrò darti la corretta ora locale. 😊'

### Langchain agents
Langhchain, sopra questi modelli, permette di creare agenti AI che possono eseguire compiti più complessi.
Possono fare ragionamenti, prendere decisioni, e interagire con strumenti esterni (tools).
Per creare un agente semplice, si usa `create_agent`.

In [6]:
from langchain.agents import create_agent
agent = create_agent(
    model=model
)


Anche qui, si deve utilizzare `invoke` per interagire con l'agente.
La differenza è che l'agente acceta una lista di messaggi (come nei modelli chat).
I messaggi hanno anche un ruolo (user, assistant, system).

In [7]:
from langchain_core.messages import HumanMessage, SystemMessage
agent.invoke({
    "messages": [HumanMessage("Che ore sone?")]
})

{'messages': [HumanMessage(content='Che ore sone?', additional_kwargs={}, response_metadata={}, id='93b342b6-05ea-4b09-950e-60137738e077'),
  AIMessage(content='La domanda "Che ore sone?" (che ore è ora?) non ha un risposta unica senza sapere la tua zona oraria. Il tempo varia in base alla tua ubicazione geografica. \n\nSe vuoi, puoi dirmi il tuo luogo o la tua zona oraria (es. Italia, Stati Uniti, Brasile, etc.), e ti aiuto ad indicare l\'ora corretta! 😊\n\n**Come verificare l\'ora corrente:**  \n- Usa un cronometro online (es. [TimeAndDate](https://www.timeanddate.com/))  \n- Controlla un\'app di navigazione o un dispositivo mobile  \n- Verifica il tuo orologio o un calendario  \n\nSe hai un\'informazione su di tua, fammi sapere! 🌍', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 361, 'prompt_tokens': 15, 'total_tokens': 376, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'qwen

Per la conversazione, si può simulare una chat mantenendo lo stato della conversazione.

In [16]:
steps = 3
messages = []
for i in range(steps):
    user_input = input("User: ")
    messages.append(HumanMessage(content=user_input))
    response = agent.invoke({"messages": messages})
    print(f"Agent: {response['messages'][-1].content}")
    messages.append(response['messages'][-1])

Agent: Sono bene, ma posso aiutarti con qualcosa?
Agent: 3 + 2 = 5. Il risultato è 5.
Agent: La data è il 11 aprile 2023, e le ore sono 14:30. C'è qualcosa che posso aiutarti oggi?


## Strumenti
Un LLM non può fare tutto da solo. Spesso ha bisogno di strumenti esterni per ottenere informazioni aggiornate o eseguire calcoli.
Questo si basa principalmente sull'idea di "tool use", dove l'agente decide quando e come usare uno strumento.
Questa cosa, può essere fatta via prompt engineering.
Si deve:
- Definire gli strumenti disponibili
- Fornire un prompt che spiega come usare gli strumenti
- Analizzare la risposta del modello per vedere se vuole usare uno strumento
- Eseguire lo strumento se necessario e fornire il risultato al modello per una risposta

In [9]:
# Tool definitions
from datetime import datetime
TOOLS = {
    "get_current_time": lambda: datetime.now().strftime("%H:%M:%S"),
    "get_current_date": lambda: datetime.now().strftime("%Y-%m-%d")
}


Dati questi tools, possiamo creare un prompt che spiega come usarli.

In [10]:
TOOL_PROMPT = """
System Prompt:
You are an AI agent that can use the following tools to assist users:
Tools:
1. Get Current Time
   - Description: Returns the current time in HH:MM:SS format.
   - Interface: get_current_time() -> str
2. Get Current Date
   - Description: Returns the current date in YYYY-MM-DD format.
   - Interface: get_current_date() -> str

When given a user request, decide if you need to use a tool. If so, generate a command in the format:
<tool_name>()
following the tool's Interface (e.g., get_current_time(), or get_current_date()).

If you use a tool, respond with ONLY the tool call in the exact format specified.
If you don't need a tool, respond normally to the user.
"""


Devo poi fare in modo che l'agente sia in grado di interpretare il prompt e chiamare i tools.
Per questo, si potrebbero usare espressioni regolari per estrarre il nome dello strumento dalla risposta del modello.

In [11]:
import re
def parse_tool_call(response: str):
    """Parse tool_name() from response"""
    if match := re.search(r'(\w+)\(\)', response):
        tool_name = match.group(1)
        return tool_name, {}
    return None


Infine, scrivo l'agente "vero e proprio" che usa i tools via prompt engineering.

In [12]:
def agent_with_tools_prompt(user_input: str) -> str:
    """Agent that uses tools via prompt engineering"""
    response = model.invoke(f"{TOOL_PROMPT}\n\nUser: {user_input}\nAssistant:").content
    print(f"LLM Response: {response}")

    if tool_call := parse_tool_call(response):
        tool_name, params = tool_call
        print(f"Tool call: {tool_name}()")

        if tool_name in TOOLS:
            result = TOOLS[tool_name]()
            print(f"Result: {result}")
            return model.invoke(
                f"{TOOL_PROMPT}\n\nUser: {user_input}\n"
                f"Tool {tool_name} returned: {result}\nProvide final answer:"
            ).content
        return f"Error: Tool '{tool_name}' not found."

    return response


In [13]:
# Esempi d'uso

print("Risultato: " + agent_with_tools_prompt("Che ore sono adesso?"))

print("Risultato: " + agent_with_tools_prompt("Qual è la data di oggi?"))

print("Risultato: " + agent_with_tools_prompt("In che mese siamo?"))

print("Risultato: " + agent_with_tools_prompt("Ciao, come stai?"))

LLM Response: <get_current_time()>
Tool call: get_current_time()
Result: 16:37:09
Risultato: 16:37:09
LLM Response: <get_current_date()>
Tool call: get_current_date()
Result: 2025-11-10
Risultato: La data di oggi è 2025-11-10.
LLM Response: <get_current_date()>
Tool call: get_current_date()
Result: 2025-11-10
Risultato: The current month is November.
LLM Response: Ciao! Sono un AI e non ho un corpo, però posso aiutarti con qualsiasi cosa necessiti. Come posso aiutarti oggi?
Risultato: Ciao! Sono un AI e non ho un corpo, però posso aiutarti con qualsiasi cosa necessiti. Come posso aiutarti oggi?


## LangChain tools
LangChain predispone nella modalità "Agent" di utilizzare direttamente i tools.
È possibile farlo sia con MCP che con funzioni python pure (via @tool).

In [14]:
from langchain.tools import tool
from datetime import datetime

@tool
def get_current_time() -> str:
    """
    Ritorna l'ora corrente nel formato HH:MM:SS
    """
    return datetime.now().strftime("%H:%M:%S")

agent_with_time = create_agent(
    model=model,
    tools=[get_current_time]
)


Una funzione annotata in questo modo, viene automaticamente convertita in uno strumento che l'agente può usare.

In [15]:
get_current_time

StructuredTool(name='get_current_time', description="Ritorna l'ora corrente nel formato HH:MM:SS", args_schema=<class 'langchain_core.utils.pydantic.get_current_time'>, func=<function get_current_time at 0x7fbef465cc20>)

Ora l'agente *può* usare il tool in modo autonomo quando ne ha bisogno

In [16]:
result = agent_with_time.invoke({
    "messages": [HumanMessage("Che ore sono?")]
})

result['messages'][2]

ToolMessage(content='16:40:07', name='get_current_time', id='b0b02330-0d3d-41c1-878f-eee79c87e4ff', tool_call_id='call_5f04lorh')

In [22]:
result['messages'][3]

AIMessage(content="L'ora corrente è 16:40:07.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 147, 'prompt_tokens': 183, 'total_tokens': 330, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'qwen3:1.7b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-669', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--2d8b93bc-c2d9-4910-bcd2-ca9ec3d743cf-0', usage_metadata={'input_tokens': 183, 'output_tokens': 147, 'total_tokens': 330, 'input_token_details': {}, 'output_token_details': {}})

È possibile, infine, basarsi su MCP per caricare tools da server remoti.
Eseguendo lo script `time-mcp.py` si avvia un server MCP che espone un tool per ottenere l'ora corrente.

In [45]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent

client = MultiServerMCPClient(
    {
        "time": {
            "transport": "streamable_http",
            "url": "http://localhost:8000/mcp",
        }
    }
)

tools = await client.get_tools() # prendo i tools dal server MCP
agent = create_agent(model=model, tools=tools)
time_response = await agent.ainvoke(
    {"messages": [{"role": "user", "content": "che ore sono??"}]}
)
# Mostra la risposta finale dell'agente in modo più leggibile
print("Risposta dell'agente:")
print(time_response['messages'][-1].content)

Risposta dell'agente:
Ist tempo corrente è **28 settembre 2025 alle 19:41** (orario locale). Se necessiti di informazioni aggiornate, puoi ripetere la domanda! 🌞


È possibile anche sfruttare MCP in locale usando Docker.
In questo esempio, usiamo `docker mcp gateway run --servers=time` per avviare un server MCP locale che espone il tool per ottenere l'ora corrente.

In [47]:
from mcp import ClientSession
from langchain_mcp_adapters.tools import load_mcp_tools
from mcp.client.stdio import stdio_client, StdioServerParameters

server_params = StdioServerParameters(
    command="docker",
    args=["mcp", "gateway", "run", "--servers=time"]
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        tools = await load_mcp_tools(session)
        agent_with_mcp = create_agent(
            model=model,
            tools=tools
        )
        result = await agent_with_mcp.ainvoke({  # Add await here
            "messages": [{"role": "user", "content": "Che ore sono? (in UTC)"}]
        })
        print(result['messages'][-1].content)

In UTC time, it's currently **October 8, 2025 at 18:44**. The time zone is set to UTC, and no daylight saving adjustments are currently in effect. Let me know if you need the time in a different timezone!
